In [ ]:
from setup import *
from agents import *
from env import *

In [ ]:

buyerStrategies = ['DDPG', 'DDPG']
sellerStrategies = ['TruthTeller', 'TruthTeller', 'TruthTeller', 'TruthTeller', 'TruthTeller', 'TruthTeller']
gamereinforcers, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed = '1001', len(buyerStrategies), len(sellerStrategies), 3, 1, 10000, 9, 42
disclosure = ['rnd', 'period', 'step','currentBid','currentAsk','buy','sell','price','sale']
gameData = [gamereinforcers, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed] 
buyers, sellers = generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure)
log = Log(gameData, buyerStrategies, sellerStrategies, disclosure)

#Hyperparameters
lr_mu        = 0.0005
lr_q         = 0.001
gamma        = 0.99
batch_size   = 32
buffer_limit = 50000
tau          = 0.005 # for target network soft update

for rnd in range(numRounds):
    roundData = roundSetup(*gameData)
    redemptionValues, tokenCosts, demand, supply, prices, peq, qeq  = roundData[0:7]
    [buyerReservationPrices, sellerReservationPrices, buyerSurplus, sellerSurplus, totalSurplus, buyerSurplusFrac, sellerSurplusFrac] = roundData[7:]
    log.addRound([rnd] + roundData)
    resetRounds(buyers, sellers, redemptionValues, tokenCosts)
    for period in range(numPeriods):
        resetPeriods(buyers, sellers)
        bids_ = []
        for step in range(numSteps):
            resetSteps(buyers, sellers)
            bids, asks = collectOffers(buyers, sellers)
            currentAsk, currentAskIdx, currentBid, currentBidIdx = bestOffers(bids, asks)
            price, buy, sell = trade(buyers, sellers, currentAsk, currentAskIdx, currentBid, currentBidIdx)
            bprofit, sprofit = 0, 0
            bids_.append(bids)
            if price > 0:
                buyers[currentBidIdx].transact(price)
                sellers[currentAskIdx].transact(price)
                bprofit = buyers[currentBidIdx].stepProfits
                sprofit = sellers[currentAskIdx].stepProfits
            log.addStep([rnd, period, step, bids, asks, currentBid, currentBidIdx, currentAsk, currentAskIdx, buy, sell, price, price>0, bprofit, sprofit])
            observe(buyers, sellers, log.disclose()) # observe new state
            updateStates(buyers, sellers)
        updatePolicy(buyers, sellers)
        
        if period % (numPeriods/1000) == 0:
            print(f'{period},{log.getPeriod(rnd, period).tail(100)[["bprofit", "currentBidIdx"]].groupby("currentBidIdx").sum().bprofit}, {np.nanmean(bids_, axis=0)}')
            #display(log.getPeriod(rnd, period).tail(100)[['sprofit', 'currentAskIdx']].groupby('currentAskIdx').sum())
            #display(log.getPeriod(rnd,period))

0,currentBidIdx
0.0    146.1
1.0     86.6
Name: bprofit, dtype: float64, [22.31666667 53.65      ]
10,currentBidIdx
0.0    144.30
1.0     89.35
Name: bprofit, dtype: float64, [ 8.7 61.7]
20,currentBidIdx
0.0    144.3
1.0    108.6
Name: bprofit, dtype: float64, [ 1.         48.86666667]
30,currentBidIdx
0.0    144.3
1.0    116.8
Name: bprofit, dtype: float64, [ 1.  43.4]
40,currentBidIdx
0.0    144.3
1.0    127.6
Name: bprofit, dtype: float64, [ 1.  36.2]
50,currentBidIdx
0.0    144.3
1.0    129.7
Name: bprofit, dtype: float64, [ 1.  34.8]
60,currentBidIdx
0.0    144.30
1.0    130.05
Name: bprofit, dtype: float64, [ 1.         34.56666667]
70,currentBidIdx
0.0    144.3
1.0    140.9
Name: bprofit, dtype: float64, [ 5.51666667 27.33333333]
80,currentBidIdx
0.0    144.3
1.0    128.0
Name: bprofit, dtype: float64, [12.7        35.93333333]
90,currentBidIdx
0.0    144.30
1.0    124.05
Name: bprofit, dtype: float64, [19.26666667 38.56666667]
100,currentBidIdx
0.0    144.3
1.0    119.5
Name: b

In [43]:
log.getPeriod(0,2450)

,rnd,period,step,bids,asks,currentBid,currentBidIdx,currentAsk,currentAskIdx,buy,sell,price,sale,bprofit,sprofit
22050,0,2450,0,"[1.0, 13.4]","[1.0, 34.6, 41.6, 0.0, 28.2, 2.8]",13.4,1.0,0.0,3,True,True,6.70,True,60.10,6.70
22051,0,2450,1,"[9.9, 51.3]","[1.0, 34.6, 41.6, 35.0, 28.2, 2.8]",51.3,1.0,1.0,0,True,True,26.15,True,40.05,25.15
22052,0,2450,2,"[8.2, 20.5]","[51.3, 34.6, 41.6, 35.0, 28.2, 2.8]",20.5,1.0,2.8,5,True,True,11.65,True,39.15,8.85
22053,0,2450,3,"[5.6, nan]","[51.3, 34.6, 41.6, 35.0, 28.2, 38.9]",5.6,0.0,28.2,4,True,False,28.20,True,71.80,0.00
22054,0,2450,4,"[7.0, nan]","[51.3, 34.6, 41.6, 35.0, 54.1, 38.9]",7.0,0.0,34.6,1,True,False,34.60,True,58.40,0.00
22055,0,2450,5,"[11.4, nan]","[51.3, 41.4, 41.6, 35.0, 54.1, 38.9]",11.4,0.0,35.0,3,True,False,35.00,True,14.10,0.00
22056,0,2450,6,"[nan, nan]","[51.3, 41.4, 41.6, 43.3, 54.1, 38.9]",NaN,NaN,38.9,5,NaN,NaN,NaN,False,0.00,0.00
22057,0,2450,7,"[nan, nan]","[51.3, 41.4, 41.6, 43.3, 54.1, 38.9]",NaN,NaN,38.9,5,NaN,NaN,NaN,False,0.00,0.00
22058,0,2450,8,"[nan, nan]","[51.3, 41.4, 41.6, 43.3, 54.1, 38.9]",NaN,NaN,38.9,5,NaN,NaN,NaN,False,0.00,0.00


In [44]:
class DDPG(Trader):
    def __init__(self, gameData, disclosure, index, buyer, reinforcer):
        super().__init__(gameData, disclosure, index, buyer, reinforcer)
        self.numActions = 100
        self.depth = 20
        self.numStates = 8 + 6 * self.depth
        history = [-1] * self.numStates
        self.state = np.nan_to_num(np.array(history, dtype = np.float32), nan=-9)  
        self.memory = ReplayBuffer()
        self.q, self.q_target = QNet(self.numStates), QNet(self.numStates)
        self.q_target.load_state_dict(self.q.state_dict())
        self.mu, self.mu_target = MuNet(self.numStates), MuNet(self.numStates)
        self.mu_target.load_state_dict(self.mu.state_dict())
        self.mu_optimizer = optim.Adam(self.mu.parameters(), lr=lr_mu)
        self.q_optimizer  = optim.Adam(self.q.parameters(), lr=lr_q)
        self.ou_noise = OrnsteinUhlenbeckNoise(mu=np.zeros(1))
        self.done = False

    def observe(self):
        agentState = [self.periodStep, self.stepTokenValue, self.stepBid, self.stepAsk, self.stepTrades, self.stepProfits, self.periodTrades, self.periodProfits]
        history = self.df.iloc[-self.depth:][['currentBid', 'currentAsk', 'buy', 'sell', 'price', 'price']].values.reshape(-1,).tolist()
        if len(history) != (6 * self.depth):
            history = [-1] * (6 * self.depth)
        self.newState = np.nan_to_num(np.array(agentState + history, dtype = np.float32), nan=-9)
        if self.periodStep == self.numSteps:
            self.done = True
        self.memory.put((self.state,self.a,self.stepProfits/100.0,self.newState,self.done))
        self.state = self.newState 

    def train(self):
        if self.memory.size()>2000:
            for i in range(10):
                train(self.mu, self.mu_target, self.q, self.q_target, self.memory, self.q_optimizer, self.mu_optimizer)
                soft_update(self.mu, self.mu_target)
                soft_update(self.q,  self.q_target)
        
    def bid(self):
        self.stepBid = np.nan
        if self.stepTokenValue > 0:
            a = self.mu(torch.from_numpy(self.state).float()) 
            self.a = a.item() + self.ou_noise()[0]
            frac = np.clip(self.a,0,1)
            self.stepBid = 1*frac + (1-frac)*100
        return np.round(self.stepBid,1)
        
    def ask(self):
        self.stepAsk = np.nan
        if self.stepTokenValue > 0:
            self.mu, self.std = self.model.pi(torch.from_numpy(self.state).float())
            dist = Normal(self.mu, self.std)
            self.action = dist.sample()
            self.log_prob = dist.log_prob(self.action)
            frac = self.action.item()
            self.stepAsk = np.round(frac * 100,1)
        return self.stepAsk

In [45]:
import gym
import random
import collections
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
lr_mu        = 0.0005
lr_q         = 0.001
gamma        = 0.99
batch_size   = 32
buffer_limit = 50000
tau          = 0.005 # for target network soft update

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask = 0.0 if done else 1.0 
            done_mask_lst.append([done_mask])
        
        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst, dtype=torch.float), \
                torch.tensor(r_lst, dtype=torch.float), torch.tensor(s_prime_lst, dtype=torch.float), \
                torch.tensor(done_mask_lst, dtype=torch.float)
    
    def size(self):
        return len(self.buffer)

class MuNet(nn.Module):
    def __init__(self, numStates):
        super(MuNet, self).__init__()
        self.fc1 = nn.Linear(numStates, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc_mu = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        mu = torch.sigmoid(self.fc_mu(x)) # Multipled by 2 because the action space of the Pendulum-v0 is [-2,2]
        return mu

class QNet(nn.Module):
    def __init__(self, numStates):
        super(QNet, self).__init__()
        self.fc_s = nn.Linear(numStates, 64)
        self.fc_a = nn.Linear(1,64)
        self.fc_q = nn.Linear(128, 32)
        self.fc_out = nn.Linear(32,1)

    def forward(self, x, a):
        h1 = F.relu(self.fc_s(x))
        h2 = F.relu(self.fc_a(a))
        cat = torch.cat([h1,h2], dim=1)
        q = F.relu(self.fc_q(cat))
        q = self.fc_out(q)
        return q

class OrnsteinUhlenbeckNoise:
    def __init__(self, mu):
        self.theta, self.dt, self.sigma = 0.1, 0.01, 0.1
        self.mu = mu
        self.x_prev = np.zeros_like(self.mu)

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
                self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        self.x_prev = x
        return x
      
def train(mu, mu_target, q, q_target, memory, q_optimizer, mu_optimizer):
    s,a,r,s_prime,done_mask  = memory.sample(batch_size)
    
    target = r + gamma * q_target(s_prime, mu_target(s_prime)) * done_mask
    q_loss = F.smooth_l1_loss(q(s,a), target.detach())
    q_optimizer.zero_grad()
    q_loss.backward()
    q_optimizer.step()
    
    mu_loss = -q(s,mu(s)).mean() # That's all for the policy loss.
    mu_optimizer.zero_grad()
    mu_loss.backward()
    mu_optimizer.step()
    
def soft_update(net, net_target):
    for param_target, param in zip(net_target.parameters(), net.parameters()):
        param_target.data.copy_(param_target.data * (1.0 - tau) + param.data * tau)
    

In [46]:
def generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure):
    buyers, sellers = [], []
    for idx,i in enumerate(buyerStrategies):
        if i == 'TruthTeller':
            buyers.append(TruthTeller(gameData, disclosure, index=idx, buyer=1, reinforcer=0)) 
        if i == 'ZeroIntelligence':
            buyers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=1, reinforcer=0)) 
        if i == 'REINFORCE':
            buyers.append(REINFORCE(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'PPO':
            buyers.append(PPO(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'SAC':
            buyers.append(SAC(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'DQN':
            buyers.append(DQN(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'DDPG':
            buyers.append(DDPG(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 

    for idx,i in enumerate(sellerStrategies):
        if i == 'TruthTeller':
            sellers.append(TruthTeller(gameData, disclosure, index=idx, buyer=0, reinforcer=0)) 
        if i == 'ZeroIntelligence':
            sellers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=0, reinforcer=0)) 
        if i == 'REINFORCE':
            sellers.append(REINFORCE(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
        if i == 'PPO':
            sellers.append(PPO(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
        if i == 'SAC':
            sellers.append(SAC(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
        if i == 'DDPG':
            sellers.append(DDPG(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
    return buyers, sellers